# Matchmaker Basic Usage

This notebook demonstrates the core matchmaker library functionality:
- Loading interaction data with gender support
- Computing engagement scores
- Computing popularity metrics and leagues
- Training ALS models for recommendations

### ⚠️ Note: NVIDIA Driver Installation Required ⚠️

Verify with: ```nvidia-smi```

In [1]:
from matchmaker import matchmaker
import cudf

/home/zain/anaconda3/envs/matchmaker-dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw = cudf.read_csv("data/swipes_clean.csv")

In [3]:
engine = matchmaker.MatchingEngine()

In [4]:
engine.load_interactions("data/swipes_clean.csv", 
  decider_col='decidermemberid',
  other_col='othermemberid', 
  like_col='like', 
  timestamp_col='timestamp',
  gender_col='decidergender')

Reading data... ✅
Constructing graph...✅
Constructing graph...✅
Fitting ALS... 🚀 Preparing data...
✅
Fitting ALS... 🚀 Preparing data...
🎯 Training male→female ALS...
🎯 Training male→female ALS...


100%|██████████| 15/15 [00:01<00:00, 11.47it/s]



🎯 Training female→male ALS...


100%|██████████| 15/15 [00:00<00:00, 218.24it/s]



🔄 Converting factors to CuPy arrays...
✅ Trained M2F ALS with 33173 males × 33358 females
✅ Trained F2M ALS with 10882 females × 44241 males
Complete! ✅
Complete! ✅


In [5]:
engine.run_engagement()

User DF updated ✅


In [6]:
engine.run_popularity()

User DF updated ✅


In [7]:
user_df = engine.user_df
interaction_df = engine.interaction_df
interaction_graph = engine.interaction_graph
als_model = engine.als_model

In [8]:
user_df.dropna().sample(5)

,user_id,gender,weighted_swipes,weighted_likes,total_swipes,total_likes,like_rate_raw,like_rate_weighted,like_rate_smoothed,decision_confidence,...,likes_received,swipes_received,weighted_likes_received,weighted_swipes_received,in_like_rate_raw,in_like_rate_weighted,in_like_rate_smoothed,popularity_confidence,popularity_score,league
16003,909415,M,93.603038,60.503130,96,62.0,0.645833,0.646380,0.517838,0.737417,...,3.0,39,2.841400,37.344268,0.076923,0.076087,0.176297,0.816853,0.249431,Silver
11737,2696215,F,32.977577,13.325223,34,14.0,0.411765,0.404069,0.364564,0.375690,...,241.0,429,230.156014,409.121965,0.561772,0.562561,0.551530,1.000000,1.000000,Diamond
10010,26606,F,31.543465,29.609460,33,31.0,0.939394,0.938688,0.530080,0.362768,...,47.0,92,44.575717,87.754318,0.510870,0.507960,0.475576,0.981479,1.000000,Platinum
29414,560959,M,270.409220,229.052567,281,238.0,0.846975,0.847059,0.744014,0.978995,...,0.0,18,0.000000,17.087829,0.000000,0.000000,0.194967,0.540088,0.000000,Bronze
22676,1550017,M,26.143983,0.000000,28,0.0,0.000000,0.000000,0.251880,0.311669,...,1.0,9,0.971989,8.708725,0.111111,0.111611,0.279817,0.326893,0.091470,Silver


In [17]:
als_model.mutual_score(male_id=1550017, female_id=743353)

9.783194400370121e-06

In [10]:
als_model.recommend_for_user(2961297)

[(3865564, 0.8798269033432007),
 (3864277, 0.7180454209446907),
 (3862214, 0.6504687424749136),
 (3683040, 0.6319559812545776),
 (1030818, 0.6156509295105934),
 (2576745, 0.6018807468935847),
 (3630607, 0.5891541242599487),
 (3707966, 0.5848696231842041),
 (3863962, 0.5802642852067947),
 (1602148, 0.5749529991298914)]

In [11]:
als_model.recommend_batch(['3176540','2636318'])

{3176540: [(3049402, 0.10849692299962044),
  (3863815, 0.0837312787771225),
  (2005913, 0.08050452545285225),
  (3863253, 0.07245548348873854),
  (3870355, 0.05768155143596232),
  (3863350, 0.05690122302621603),
  (3359195, 0.05553269758820534),
  (3804841, 0.05541134066879749),
  (510339, 0.05414690263569355),
  (3144618, 0.05356806190684438)],
 2636318: [(3359195, 0.8327882073936053),
  (3144618, 0.8306626521516591),
  (510339, 0.814813530741958),
  (3863376, 0.7620853792177513),
  (3632714, 0.7425037362845615),
  (3864112, 0.7408705675043166),
  (3873550, 0.7126078737492207),
  (3804841, 0.7054699240252376),
  (3877191, 0.6506750393891707),
  (3873194, 0.6167885403847322)]}

In [12]:
als_model.get_allowed_ids()

{'male_ids': [338207,
  174471,
  1495746,
  833576,
  267676,
  259471,
  3508114,
  3858399,
  2078912,
  344021,
  1624195,
  430672,
  3264338,
  221970,
  3863879,
  347364,
  393262,
  2958241,
  238368,
  3235303,
  1829188,
  687724,
  3688476,
  1469055,
  50327,
  758121,
  982122,
  3418884,
  353816,
  1244503,
  1301850,
  2597170,
  98533,
  451126,
  1798988,
  2057609,
  58829,
  2885255,
  253210,
  3851137,
  202395,
  494468,
  3840858,
  277354,
  2973262,
  163840,
  370737,
  679634,
  468640,
  3651709,
  3859701,
  3138143,
  6159,
  2559318,
  86306,
  3849921,
  1782934,
  152605,
  1394608,
  583230,
  2465307,
  232674,
  627139,
  898953,
  2738889,
  3734966,
  556480,
  107018,
  300315,
  3780279,
  230987,
  3859735,
  1186323,
  3811489,
  1545005,
  165971,
  3487075,
  2043461,
  3808410,
  733230,
  520081,
  250070,
  3834492,
  232193,
  498532,
  713451,
  3583783,
  1463144,
  490673,
  181784,
  2947555,
  382725,
  2008579,
  1632080,
  164337

In [13]:
print('Number of males in ALS:', len(als_model.male_map))
print('Number of females in ALS:', len(als_model.female_map))

# Check if specific IDs are actually in the map
print('445950 in male_map?', '445950' in als_model.male_map)
print('743353 in female_map?', '743353' in als_model.female_map)

Number of males in ALS: 33173
Number of females in ALS: 33358
445950 in male_map? False
743353 in female_map? False


In [14]:
print('445950 in male_map?', 445950 in als_model.male_map)
print('743353 in female_map?', 743353 in als_model.female_map)

445950 in male_map? True
743353 in female_map? True


## ✅ Summary of Fixes

The ALS implementation now works correctly with the following fixes:

1. **Factor Conversion**: Properly converts implicit's numpy arrays to CuPy with explicit float32 dtype
2. **Separate Model Mappings**: M2F and F2M models maintain their own ID mappings since they're trained on different user sets
3. **Graceful Score Handling**: `mutual_score()` returns partial scores when only one direction is available, returns `None` only when neither model has data
4. **Filtered Recommendations**: `recommend_for_user()` filters out None scores before sorting

**Model Stats:**
- M2F Model: ~33K males → ~33K females  
- F2M Model: ~11K females → ~44K males  
- Different coverage is normal - not all users have outbound interactions in both directions